<a href="https://colab.research.google.com/github/linztjavier-max/BASC0005-London-Air-Inequality/blob/main/2019_earnings_v3_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# data in earnings
df_path_earnings = "https://data.london.gov.uk/download/2z0rk/1686ef1c-b169-442d-8877-e7e49788f668/earnings-residence-borough.xlsx"

earnings_df = pd.read_excel(df_path_earnings, sheet_name="Total, weekly")

In [ ]:
import pandas as pd

raw_df = pd.read_excel(df_path_earnings, sheet_name="Total, weekly", header=None)

# Extract the first two rows to be used as header information
header_row0 = raw_df.iloc[0] # Contains years (e.g., 2002, NaN, 2003, NaN)
header_row1 = raw_df.iloc[1] # Contains sub-headers (e.g., Code, Area, Pay (£), conf %)

# Construct new column names by combining the year and sub-header
new_columns = []
current_year = None

for i in range(len(header_row0)):
    year_val = header_row0.iloc[i]
    sub_header_val = header_row1.iloc[i]

    if i < 2: # Handle the first two columns ('Code', 'Area') specifically
        new_columns.append(str(year_val).strip())
    elif pd.isna(year_val): # If year is NaN, it's a sub-header like 'conf %' under a year
        if current_year is not None:
            new_columns.append(f"{current_year} {str(sub_header_val).strip()}")
        else:
            # This case implies a NaN year_val without a preceding year, which shouldn't happen for data columns
            new_columns.append(str(sub_header_val).strip()) # Fallback for safety
    else: # Year value is present (e.g., 2002, 2003, ...)
        current_year = int(year_val)
        new_columns.append(f"{current_year} {str(sub_header_val).strip()}")

# Create the earnings_clean DataFrame by taking data from the third row onwards
# and assigning the newly constructed column names.
earnings_clean = raw_df.iloc[2:].copy()
earnings_clean.columns = new_columns
earnings_clean = earnings_clean.reset_index(drop=True)

# Remove all 'conf %' columns
columns_to_drop = [col for col in earnings_clean.columns if 'conf %' in col]
earnings_clean = earnings_clean.drop(columns=columns_to_drop)

# Identify the columns for 'Pay (£)' for years 2011 to 2024
years_to_keep = list(range(2019, 2023))
pay_columns = [f"{year} Pay (£)" for year in years_to_keep]

# Ensure 'Code' and 'Area' are always kept
final_columns = ['Code', 'Area'] + pay_columns

# Filter earnings_clean to retain only these selected columns
earnings_clean = earnings_clean[final_columns]

earnings_clean_df = earnings_clean.iloc[0:34]

print("Cleaned earnings_clean DataFrame:")
earnings_clean_df

In [ ]:
earnings_df_2019=earnings_clean_df[["Code", "Area", "2019 Pay (£)"]]
earnings_df_2019

In [ ]:
#choropleth 2019 income
#import necessary libraries

import warnings

import geopandas as gpd
import libpysal as lps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import esda

In [ ]:
geojson_url = "https://hub.arcgis.com/api/v3/datasets/0a92a355a8094e0eb20a7a66cf4ca7cf_10/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1"
output_filename = "london_boroughs.geojson"

In [ ]:
import requests

# Download the GeoJSON file
response = requests.get(geojson_url)
response.raise_for_status()  # Raise an exception for bad status codes

with open(output_filename, 'wb') as f:
    f.write(response.content)

gdf = gpd.read_file(output_filename)

In [ ]:
gdf

In [ ]:
np.random.seed(12345)
import esda

In [ ]:
earnings_df_2019_borough = earnings_df_2019.rename(columns={'Area': 'Borough'})

In [ ]:
earnings_df_2019_borough = earnings_df_2019_borough.rename(columns={'Code': 'LA_Code'})

In [ ]:
#Standardise borough names: inspect the unique values in the 'Borough' column
earnings_df_2019_borough['Borough'].unique()

In [ ]:
#Inspect the unique values in the 'BOROUGH' column of the gdf DataFrame to compare them with the unique borough names from vehicles_df_2019_borough and identify any inconsistencies.
gdf['BOROUGH'].unique()

In [ ]:
#edit names where necessary in vehicles df to match gdf
earnings_df_2019_borough['Borough'] = earnings_df_2019_borough['Borough'].replace({
    'Barking and Dagenham': 'Barking & Dagenham',
    'Hammersmith and Fulham': 'Hammersmith & Fulham',
    'Kensington and Chelsea': 'Kensington & Chelsea'
})

print("Unique borough names in earnings_df_2019_borough after standardization:")
print(earnings_df_2019_borough['Borough'].unique())

In [ ]:
#Confirm all naming inconsistencies have been resolved
diff_earnings_not_in_gdf = set(earnings_df_2019_borough['Borough'].unique()) - set(gdf['BOROUGH'].unique())
diff_gdf_not_in_earnings = set(gdf['BOROUGH'].unique()) - set(earnings_df_2019_borough['Borough'].unique())

print("Borough names in earnings_df_2019_boroughh but not in gdf:", diff_earnings_not_in_gdf)
print("Borough names in gdf but not in earnings_df_2019_borough:", diff_gdf_not_in_earnings)

In [ ]:
#remove nan from earnings df
earnings_df_2019_borough = earnings_df_2019_borough.dropna(subset=['Borough'])

regions_to_remove = [
    'nan'
]

earnings_df_2019_borough = earnings_df_2019_borough[~earnings_df_2019_borough['Borough'].isin(regions_to_remove)]

print("Unique borough names in earnings_df_2019_borough after removal:")
print(earnings_df_2019_borough['Borough'].unique())

In [ ]:
#merge the data frames by using left merge (borough names as keys), then remove extra borough column
gdf_merged_2019 = gdf.merge(earnings_df_2019_borough, left_on='BOROUGH', right_on='Borough', how='left')
gdf_merged_2019 = earnings_df_2019_borough.drop(columns=['Borough'])
gdf_merged_2019.head()

In [ ]:
#Check for missing values in merged '2019 Pay (£)' column of the gdf_merged_2019 DataFrame using the .isnull().sum() method.
gdf_merged_2019['2019 Pay (£)'].isnull().sum()

In [ ]:
import pandas as pd

# The error occurs because gdf_merged_2019 is a pandas DataFrame, not a GeoDataFrame,
# which is required by gpd.sjoin for spatial predicates like "intersects".
# The previous cell incorrectly overwrote the GeoDataFrame resulting from the merge
# with a plain DataFrame.

# To fix this, we should perform a non-spatial merge between the gdf (GeoDataFrame)
# and the earnings data (earnings_df_2019_borough DataFrame) to combine the attributes
# based on borough names. This will create a new GeoDataFrame with the earnings data.
sj_gdf = gdf.merge(
    earnings_df_2019_borough,
    left_on='BOROUGH',
    right_on='Borough',
    how='left'
)

# Convert the '2019 Pay (£)' column to numeric, coercing errors to NaN
sj_gdf['2019 Pay (£)'] = pd.to_numeric(sj_gdf['2019 Pay (£)'], errors='coerce')

# Drop the redundant 'Borough' and 'LA_Code' columns from the merged GeoDataFrame
sj_gdf = sj_gdf.drop(columns=['Borough', 'LA_Code'])

sj_gdf.head()

In [ ]:
#Calculate mean earnings for each borough using the 2019 Pay (£) column and group the results by BOROUGH from the sj_gdf GeoDataFrame, storing the result in mean_earnings_gb
mean_earnings_gb = sj_gdf.groupby('BOROUGH')['2019 Pay (£)'].mean()
mean_earnings_gb.name = '2019 Earnings mean'
mean_earnings_gb

In [ ]:
#Merge calculated mean earnings for each borough (mean_earnings_gb) back into the gdf GeoDataFrame. This will associate the earnings data with the geographical features, which is essential for spatial analysis and visualization.
gdf = gdf.merge(mean_earnings_gb, left_on='BOROUGH', right_on='BOROUGH', how='left')
gdf.head()

In [ ]:
#Create choropleth map using gdf DataFrame, specifying plot size, column mapping, color scheme, legend, axis removal, and title
import matplotlib.pyplot as plt
import mapclassify as mc

fig, ax = plt.subplots(1, figsize=(12, 10), subplot_kw={'aspect': 'equal'})
gdf.plot(column='2019 Earnings mean',scheme='Quantiles', k=5, cmap='OrRd', legend=True, ax=ax)
ax.set_axis_off()
plt.title('2019 Earnings Across London Boroughs')
plt.show()

In [ ]:
#Create choropleth map using gdf DataFrame, specifying plot size, column mapping, color scheme, legend, axis removal, and title
import matplotlib.pyplot as plt
import mapclassify as mc

fig, ax = plt.subplots(1, figsize=(12, 10), subplot_kw={'aspect': 'equal'})
gdf.plot(column='2019 Earnings mean', cmap='OrRd', legend=True, ax=ax)
ax.set_axis_off()
plt.title('2019 Earnings Across London Boroughs')
plt.show()